### 第一章 Node简介

一个Node进程只允许一个js线程。多个Node进程通过通信协议协作，构成一个大型网络应用，每一个Node进程都构成这个网络应用中的一个节点，这是Node名字的本意。

v8 js引擎，webkit布局引擎

js作为一门图灵完备的语言，长久以来被禁锢在浏览器的沙盒中，js的能力取决于浏览器中间层提供的支持有多少。沙盒的外面是网卡，硬盘，显卡。。。

软件Node和chrome是对等物。他们实现的都是基于事件的异步架构。

基于事件的异步

chrome的事件服务view/dom，Node的事件服务io。

chrome中大量CPU计算会导致界面卡顿。Node中大量CPU计算会导致io卡顿。常规做法是计算和io分离，chrome把大计算交给独立线程webworker。Node中大计算交给独立线程child_process。

Node中并行io操作对程序员很方便自然基本

在多个独立的事件中，如何沟通信息

单线程的最大好处是不用像多线程编程那样处处在意“状态”同步。没有锁，没有线程上下文的切换。

Node内部基于libuv实现跨平台。libuv称为平台层，位于Node程序上层和操作系统之间的位置。libuv封装的功能包括事件循环/文件操作

### 第二章 模块机制

html5规范属于web2.0的范畴。commonjs规范的目的是让js具备开发大型应用的基础能力，而不是停留在小脚本程序的阶段，比如后端程序/客户端程序/命令行程序。commonjs规范涵盖模块系统/二进制处理/Buffer/字符集编码/流/进程调用/文件系统fs/套接字/单元测试/包管理。

w3c只负责浏览器中的bom和dom

Node由commonjs规范和es规范。

浏览器缓存js文件，Node缓存模块文件执行后的内存对象

打印一下module.paths

对于没有后缀名的模块引入，Node会尝试以js/json/node/目录类型识别。对于目录，先尝试执行目录中packagejsonMain属性指定的文件，若没有则尝试执行index.js/json/node文件.

一个文件 === 一个模块 === 一个函数 === 对象。js/json/node后缀外的文件以js方式处理。模块执行后缓存在在Module._cache对象中以文件路径为key

每个模块都首尾包装为以exports/require/module/__filename/__dirname为输入的函数。该函数被runInThisContext调用。调用后，module.exports被返回给引入方。函数输入参数中的exports === module.exports

自带模块可能是纯js模块，也可能包含一些c++代码（如fs/buffer/os模块）。自带模块中的c++代码，在Node被编译时，一起被编译为二进制。Node初始化时该二进制被放进内存。
自带模块的js源码放在lib文件夹下，c++源码放在src文件下。在编译c++前，会把lib下js源码以字符串数组形式放入c++代码中（使用js2c.py）。Node初始化时，该js字符串数组还原成字符串放进内存，被引用时直接从内存中取（由process.binding('natives')），进行头尾包装成函数后被调用，不像第三方模块从硬盘中取。执行完后缓存在NativeModule._cache对象中。

Node中数字只有double类型。在转码/编码的应用中会频繁需要位运算。第三方c++模块先编译为node文件(先后执行node-gyp configure和node-gyp build在build/release下生成node文件)，之后在import该node文件时通过dlopen()函数调用执行。node文件在windows下就是dll文件，在linux下就是so文件

Node的源码是通过gyp编译的。

linux g++/gcc === window vc++

包名时允许点下划线中划线的

npm i <含package.json的文件夹>
npm i url

package.json的scripts是用来定义钩子函数的，如key为install，在npm install时自动执行

commonjs和cmd是两码事

### 第三章 异步IO

API有CPU API，IO API。IO API有硬盘IO API，网络IO API等。IO操作往往是把非内存数据带到内存的操作。★★★

前端编程属于GUI编程的一种

客户端 === 发起请求端。服务端 === 响应请求端。Node可以做客户端也可以做服务端。

同步要求执行某一API的同时不执行其他API，即要求前一个API阻塞后一个API。
异步则没有这样的要求，每个API的执行时独立的互不干扰。在js中，在调用网络IO API的同时还能执行js/UI API，所以这种网络IO API是异步的。异步 === 独立。同步 === 有序。阻塞就可以简单除暴的实现同步效果

操作系统底层提供了利用信号回调机制的异步 IO API。linux中称为AIO

调用API就是使用计算机资源，计算机资源有IO(输入输出设备)和CPU。绝大部分的API都是CPU API。应用程序依赖操作系统暴露的IO API和CPU API进行工作。程序理论上可以在同一时刻同时使用IO和CPU，即理论上程序可以在执行IO API的同时执行CPU API。但应用程序中更常见的是使用操作系统暴露的“在使用IO时不允许使用CPU”的API，即阻塞API。

操作系统分别提供了阻塞IO和非阻塞IO的API。阻塞IO API使应用程序即使拿到CPU执行分片也啥都不执行。

linux将所有硬件IO设备表示为文件。应用程序需要进行IO操作，是先获取文件符，再利用文件符进行IO操作。可以认为，文件符是操作系统暴露给应用程序的API

以前，调用非阻塞API的应用程序，需要向操作系统轮询操作是否完成。轮询文件符 === 轮询IO操作。拿到CPU执行分片，调用阻塞API的应用程序阻塞而白白浪费该分片，调用非阻塞API的应用程序忙于轮询也消耗浪费该分片。调用epoll API的应用程序休眠而拿不到该分片。

轮询API中read最原始，poll和select类似，epoll最先进。epoll === 事件通知执行回调的机制。

程序的执行是消耗CPU执行分片的。一段程序调用API的数量时确定的，消耗的分片数量是确定的★

node中的异步IO效果不是简单调用linux的AIO API实现的。而是开启了新的线程，在该线程上调用操作系统的阻塞API或epoll API进行IO请求操作，当操作系统IO完成后该线程继续执行，将数据返回给主线程，从而实现异步IO效果。简而言之，node是通过线程池+阻塞IO模拟异步IO。

window下自带“线程池+阻塞IO”模拟的异步IO实现，即IOCP。node在linux下使用自研的异步IO实现，在window直接使用IOCP，在libuv层完成该兼容性判断

node的事件循环即while循环，每次循环查看是否还有事件，有事件，则取出一个事件，查看是否有关联回调，就有执行它们。

node主循环 === 事件循环 === tick

核心模块调内建模块，内建模块调内核API。以fs.open为例，node在libuv的uv_fs_open函数中会new一个FSReqWrap传入windowsIOCP的QueueUserWorkItem API，将其交给IOCP等待分配线程池进行执行。这个请求对象封装了请求的方法/参数/回调。无论这个请求方法是阻塞还是非阻塞的★，调用QueueUserWorkItem后fs.open立即返回。这样，js线程继续执行后续其他操作，io操作在IOCP中等待，互相独立，达到异步的效果。简而言之，就是将请求交由IOCP，IOCP会利用线程池已有的线程，在该线程上执行阻塞或非阻塞IO，IO借宿后将IO返回值放入FSReqWrap对象，并调用PostQueuedCompletionStatus告知IOCP IO结束。

node主循环每次tick时都获取IOCP中已完成的IO请求（通过GetQueuedCompletionStatus）的请求对象加入IO观察者队列，然后请求对象作为事件取出其回调函数执行，如图p59。

window中由内核中的IOCP来负责处理应用需要的IO请求，并负责让应用查询已完成的IO请求。linux中node基于epoll API自研了基于多线程的异步IO实现。

node中存在一些与IO操作无关的异步API，这些异步API好像是阻塞的。包含setTimeout/setInterval和setImmediate/process.nextTick

非阻塞 ===  并行。非阻塞IO是指与其他的IO操作，与其他的CPU操作可以同时做。

调用setTimeout/setInterval会new出定时器对象，是定时器观察者在管理这些对象。定时器观察者内部维护一颗红黑树。每次Tick执行时，从红黑树迭代出定时器对象，判断是否超过定时时间，如果超过，就产生一个事件并就地执行回调。★★★如果上一个Tick执行时长过长，则该定时器回调无法准时执行

process.nextTick比setTimeout(0)轻量，因为前者只使用队列数组，而后者创建定时器对象使用红黑树都更消耗性能。队列内的回调会在下次Tick时全部执行。

每次Tick中会对不同观察者进行检查。检查是由先后次序的。idle观察者先于IO观察者先于check观察者。process.nextTick时idle观察者，优先级最高。setImmediate是check观察者，使用的是链表，每个Tick只执行一个链表首部的回调。★★★所以，process.nextTick是微任务？setImmediate是宏任务，宏任务是较重的任务所以每次只执行一个。

node监听IO请求初始化结束后，就是处理IO回调了。就像chrome监听click事件初始化结束后，就是处理click回调了。

node的事件循环eventLoop === 在Tick中触发事件

有异步IO，必有异步编程

事件回调机制是基于高阶函数这一基础设施来完成的

可以批量创建函数的函数，如工厂一样创建函数的函数，称为偏函数

可以在node中利用C++拓展调用内核API

在分布式中，各个单点之间保持并行非阻塞，在这样的组织方式下，执行任务速度更快。

事件循环的这种调度方式，是在Tick中请求IO操作/处理IO操作返回的结果，在线程池中执行IO操作。Tick是CPU密集型的，如果Tick过于忙碌，无法及时请求IO操作而线程池中线程闲置，造成CPU和IO操作同步/阻塞，那么整体的处理效率降低。建议一次Tick中CPU计算量不超过10ms，否则就使用setImmediate对CPU计算进行分片。

trycatch只能捕获当前事件循环/Tick中的异常

可以说，setTimeout就是sleep函数。

cluster模块基于child_process模块 

事件监听/发布订阅模式是一种广泛存在于异步编程的模式，是回调函数的“事件化”。

hook/钩子函数，hook的是内部状态。

试试emitter.setMaxListener(0)将监听函数数量限制解除。

对于emitter对象，如果error事件被监听了，则遇到报错是执行该事件回调。没有监听该事件才抛出异常。我们应该监听error事件，提高node线程健壮性。

雪崩问题指当高访问高并发时期缓存失效，大量请求涌入数据库，超过数据库限值。比如在高访问高并发时期启动程序，此时缓存为空。

事件，函数在某确定时机执行，emitter.once('whenResponse'，action)API，场景有使同一sql语句同一时间只进行一个请求，等到这个请求成功时emitter.emit('whenResponse')也让其他同样sql请求成功。emitter.emit('whenResponse')埋点，指定实际执行时机。emitter.once('whenResponse'，action)是指定名义上的执行时机。

emit和on可以不一一对应，一个on对应多个emit，可以在on中为函数定义在多时机同时满足下执行。emit只是所有埋点中的一个。

error函数，在error时机时执行，所以，通过emitter.emit('error')在error时埋点。

事件 === 埋点执行的机制。

Promise所代表的那种模式在jQuery时代就被广为使用:
$.get('/api') // 把异步操作封装成一个异步操作对象/Deferred，这个对象内含一个事件对象/Promise，可以在这个内含对象上定义事件，通过暴露 success API 或 then API。
.success()
.success()
.error()
.complete()

Promise模式从2009年开始被抽象出来，陆续有A，B，D草案提交给CommonJS规范，使得异步IO操作的代码书写可以优雅。

```
const Promise = function(){
    EventEmitter.call(this)
}

```

$.get('/api').success或promise.then方法中为异步操作（对象）定义事件：
```
this.once('whenSuccess',fulfillFunc)
```

Deferred拥有状态域和观察者域/promise。原生异步操作顺利结束后，事先与其构成绑定关系的Deferred将状态掰成fulfilled并调用resolve事件。这种绑定关系是通过在原生对象的事件定义中调用Deferred对象的事件来达到的。

Deferred的本质只是一个观察者对象，通过resolve/reject方法让外部能够调用其事件，通过promise属性的then方法让外部能够定义其事件。而原生异步操作API本身也是一个观察者对象，但只允许外部定义其事件。★★promise模式的本质是一个中间层，之前程序是：原本定义在原生对象上的事件，现在定义在promise对象上，原本异步操作结束后，调用原生对象上的事件，现在调用promise对象上的事件。有了中间层就有了想象空间，就可以做一些事情，比如链式promise。通过判断调用事件后的返回值，如果也是一个promise对象，那么接下来的事件定义，转移到这个新的promise对象上。

在最初的promise规范的Deferrred实现中，除了resolve/reject事件还有progress事件。

p88提供了一个Promise.all的实现，具体实现是new一个deferred，维护一个count计数，在传入的每个promise上调用then定义事件：count--，当count === 0时让deferred调用事件，并返回deferred.promise让外部能够定义事件。

promise.resolve调用事件时,逐个执行事件回调队列里的函数，如果回调返回promise对象，则队列里剩下的回调转移给它

next == emit

如果对文件系统进行大量并发调用，操作系统的文件描述符的数量会被瞬间用光。通过queue对象进行过载保护。queue对象初始化时设置最大并发量，并暴露出push方法。当调用push方法时函数入队/调用_next。而_next方法中，如果当前并发量active小于最大并发量limit，调用队列中等待的第一个函数，否则不执行。而一旦有函数结束，在该函数的回调中active--/调用_next。如果调用push时内部队列长度_qlength已满，则直接报错。如果拒绝模式调用push时已到最大并发量，则直接报错，满足实时性方面的需求。

超时的IO操作也会返回，只是不会返回给程序员。在IO操作发出的那一刻会同时开启一个倒数任务，倒数任务先触发回调则意味着超时。

### 第五章 内存控制

程序需要长时间运行，才需要内存控制

电脑的唯二资源CPU和IO，通过编程语言如node利用。

js的所有对象都是堆对象，在heap中生存。试试process.memoryUsage()

node一开始会向内存申请一定的内存，当已申请的内存不够new新的对象时再向内核申请，直到达到极限直接程序崩溃。内存会有最大值是因为内存大小与GC时间正相关。1.5G的堆内存，小的GC一次50ms，一次全量GC需要1s。GC时js线程挂起。node内存最大值是2G不到。

不同对象的生命周期长度有较大差异。node内存大小 = old-space + new-space。老生代空间最大1400MB，新生代空间最大32MB。差了40倍。

新生代空间垃圾回收使用的是Scavenge/Cheney算法，该算法发表再1970年的ACM上。原理是把内存空间分成From/To两个区域(两个semispace空间)。垃圾回收就是检查From中的存活对象复制到To空间，然后From/To空间的角色对换。Scavenge的特点是用空间换时间。更细节的，在一次Cheney垃圾回收的过程中From中的存活对象可能复制到To空间，可能复制到老生代空间。

一个新对象会出生在From空间，经历过一次Scavenge回收/去过一个To空间，第二次经历Scavenge回收时就会被转移到老生代。或者如果Scavenge回收时From空间已经超过4MB则把From的所有对象送往老生代空间。